# Baseline Python version of bihar_49_band_datacube_gee.js script

This notebook is the first translation of the bihar_49_band_datacube_gee.js Earth Engine script into python. The primary benefit of such is that working in an Python environment allows for usage of pandas, numpy, and other libraries we've been using to date. On the other hand, working within the Earth Engine envirnment provides far more interactive mapping responses.

In [1]:
import ee # the earthengine libary
from IPython.display import Image # not essential but just for kernel 3's use to make sure we're good

In [2]:
ee.Initialize()

In [3]:
# this is just a check to make sure ee.Initialize() worked
# Generate a URL that displays a global DEM
url = ee.Image("USGS/SRTMGL1_003").getThumbUrl({'min':0, 'max':3000})

# Display the image in the notebook.
Image(url=url)

In [7]:
# Here's the main script that makes the 49-band datacube of Bihar

# Set the region of interest
# Asset available at https:#code.earthengine.google.com/?asset=users/bl/Ind_admin_shapefiles/ind_states
ind = ee.FeatureCollection('users/bl/Ind_admin_shapefiles/ind_states');
br = ind.filter(ee.Filter.eq('st_name','Bihar'));
roi = br;
irr_band = ee.Image("users/bawong/bassConnections/giam_2014_2015_v2")

#=================== First 11 bands to be greenest cloud free landsat imagery ===================
# cloud mask code from: https:#developers.google.com/earth-engine/ic_composite_mosaic
# This function masks clouds in Landsat 8 imagery.
def maskClouds(image):
  scored = ee.Algorithms.Landsat.simpleCloudScore(image)
  return image.updateMask(scored.select(['cloud']).lt(20))


# This function masks clouds and adds quality bands to Landsat 8 images.
def addQualityBands (image):
  return maskClouds(image).addBands(image.normalizedDifference(['B5', 'B4']))

# Load a 2014 Landsat 8 ImageCollection.
# Map the cloud masking and quality band function over the collection.
collection = ee.ImageCollection('LANDSAT/LC8_L1T_TOA').filterDate('2016-01-01', '2016-12-31').filterBounds(roi).map(addQualityBands);

# Create a greenest pixel composite.
greenestCloudfreeComposite = collection.qualityMosaic('nd').clip(roi).select(['B1','B2','B3','B4','B5','B6','B7','B8','B9','B10','B11'])


#=================== Initialization: ndvi / green index ===================
# landsat 8 imagery collection temporally bounded by month, spatially bounded by roi
ls8_base = ee.ImageCollection('LANDSAT/LC8_L1T_TOA').filterBounds(roi);
ls8 = ls8_base.filterDate('2016-01-01', '2016-01-31');
# reduce the collection to a single image by min (to minimize cloud coverage) values of each pixel
img_temp = ls8.reduce(ee.Reducer.min()).clip(roi);
img = img_temp.select(['B1_min','B2_min','B3_min','B4_min','B5_min','B6_min','B7_min','B8_min','B9_min','B10_min','B11_min'],
	 					  ['B1','B2','B3','B4','B5','B6','B7','B8','B9','B10','B11']);
# ndvi base, namely its values for jan. Other months' data will be appended later in a loop
ndvi_final = img.normalizedDifference(['B5', 'B4']).float().rename('01NDVI');
# green index base, namely its values for jan. Other months' data will be appended later in a loop
green_final = img.select('B5').divide(img.select('B3')).float().rename('01GREEN');

#=================== Initialization: rainfall data ===================
# rainfall data collection temporally bounded by month, spatially bounded by roi
GPM_raw = ee.ImageCollection('NASA/GPM_L3/IMERG_V04').filterBounds(roi);
GPM = GPM_raw.filterDate('2016-01-01', '2016-01-31');
# sum them up for the monthly rainfall
img_temp = GPM.reduce(ee.Reducer.sum()).clip(roi);
img = img_temp.select(['IRprecipitation_sum'],['IRprecipitation_double']);
# rainfall data base, namely its values for jan. Other months' data will be appended later in a loop
rain_final = img.select('IRprecipitation_double').divide(2).float().rename('01RAIN');

# add other months' ndvi, green index, and rainfall data
#for (i = 2; i<=12; i ++) {
for i in range(2, 12):
    # get it for the year of 2016, and construct a string for the start and end day of the particular month
    year = 2016
    #dayEnd  = new Date(year, i, 0).getDate();
    #dayEnd  = Date(year, i, 0).getDate();
#     monthString 
#   if (i<10){ monthString = '0'+i} else {monthString = i}
    if i < 10:
        monthString = '0'+ str(i)
    else:
        monthString = str(i)
    if any(i == _ for _ in [3,  5, 7, 10, 12]):
        dayEnd = '31'
    if any(i == _ for _ in [4,  6, 8, 9, 11]):
        dayEnd = '30'
    if i == 2:
        dayEnd = '29'
    date_start = str(year)+'-'+str(monthString)+'-01';
    date_end = str(year)+'-'+str(monthString)+'-'+str(dayEnd);

#=================== Data for the month: ndvi/green index ===================
    ls8 = ls8_base.filterDate(date_start, date_end);
    img_temp = ls8.reduce(ee.Reducer.min()).clip(roi);
    img = img_temp.select(['B1_min','B2_min','B3_min','B4_min','B5_min','B6_min','B7_min','B8_min','B9_min','B10_min','B11_min'],
                          ['B1','B2','B3','B4','B5','B6','B7','B8','B9','B10','B11']);
    ndvi = img.normalizedDifference(['B5', 'B4']).float().rename(monthString+'NDVI');# print('ndvi',ndvi)
    green = img.select('B5').divide(img.select('B3')).float().rename(monthString+'GREEN');
    
#=================== Data for the month: rainfall data ===================
    GPM = GPM_raw.filterDate(date_start, date_end);
    img_temp = GPM.reduce(ee.Reducer.sum()).clip(roi);
    img = img_temp.select(['IRprecipitation_sum'],['IRprecipitation_double']);
    rain = img.select('IRprecipitation_double').divide(2).float().rename(monthString+'RAIN');

#=================== Composing (append to each base) ===================
    ndvi_final = ndvi_final.addBands(ndvi)
    green_final = green_final.addBands(green)
    rain_final = rain_final.addBands(rain)
    irr_ag = irr_band.clip(roi).rename('irr_ag')


#=================== Add annual viirs data (stray light corrected) ===================
viirs = ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG').filterDate('2016-01-01', '2016-12-30').reduce(ee.Reducer.median()).clip(roi).select('avg_rad_median').float().rename('VIIRS2016');
     # annual composite # reduce by median

# add all bands together. Order-> 1:12=ndvi, 13:24=green index, 25:36=rainfall, 37=viirs
all_final = greenestCloudfreeComposite.addBands(ndvi_final).addBands(green_final).addBands(rain_final).addBands(viirs).addBands(irr_ag)
print(all_final.getInfo())

# If you want to visualize certain bands
#Map.addLayer(all_final.select('VIIRS2016'))

{'type': 'Image', 'bands': [{'id': 'B1', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'crs': 'EPSG:4326', 'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0]}, {'id': 'B2', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'crs': 'EPSG:4326', 'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0]}, {'id': 'B3', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'crs': 'EPSG:4326', 'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0]}, {'id': 'B4', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'crs': 'EPSG:4326', 'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0]}, {'id': 'B5', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'crs': 'EPSG:4326', 'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0]}, {'id': 'B6', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'crs': 'EPSG:4326', 'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0]}, {'id': 'B7', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'crs': 'EPSG:4326', 'crs_transform': [1.0, 0.0, 0.0, 0.